In [15]:
import numpy as np
from dnnbrain.dnn import io
import os
from PIL import Image 
import sys
sys.path.append("..")
from cnnface.stimulus.Image_process import average_img

csv_file = 'D:\cnnface\Emotion_analysis/noiseface.csv'
index_f = np.loadtxt('D:\cnnface\Emotion_analysis\CI_analysis/label_happy.txt').astype('int64')
index_m = np.loadtxt('D:\cnnface\Emotion_analysis\CI_analysis/label_sad.txt').astype('int64')

savepath_aver_f = 'D:\cnnface\Emotion_analysis\CI_analysis\ci/aver_happy_noise.jpg'
savepath_aver_m = 'D:\cnnface\Emotion_analysis\CI_analysis\ci/aver_sad_noise.jpg'
savepath_CI = 'D:\cnnface\Emotion_analysis\CI_analysis\ci/ci_noise.jpg'

#Get pictures path from stim_csv and label txt.
picset= io.PicDataset(csv_file)
pre_picpath = picset.picpath
sub_picpath = picset.picname

f_picpath = [os.path.join(pre_picpath,sub_picpath[i]) for i in index_f]
m_picpath = [os.path.join(pre_picpath,sub_picpath[i]) for i in index_m]

#average all of the pictures as their label.
aver_img_f = average_img(f_picpath)
aver_img_m = average_img(m_picpath)

#calculate the CI and superimpose image and subtract image.
ci = aver_img_f.astype('int64') - aver_img_m.astype('int64')
#ci = ci - ci.min()
ci = ci * 5
print(ci.max(),ci.min())

# (Image.fromarray(aver_img_f.astype('uint8'))).save(savepath_aver_f,quality=95)
# (Image.fromarray(aver_img_m.astype('uint8'))).save(savepath_aver_m,quality=95)
# (Image.fromarray(ci.astype('uint8'))).save(savepath_CI,quality=95)

30 -20


In [2]:
# savepath_CI = 'D:\cnnface\Emotion_analysis\CI_analysis\ci/ci_noise_nor.jpg'
# nor = lambda x:(x-x.min())/(x.max() - x.min()) 
# ci  = nor(ci) * 255
# (Image.fromarray(ci.astype('uint8'))).save(savepath_CI,quality=95)

In [34]:
base_face = np.array(Image.open(r'D:\cnnface\Emotion_analysis\face_template/baseface.jpg')).astype('int64')

ci =  np.array(Image.open('D:\cnnface\Emotion_analysis\CI_analysis\ci/ci_noise.jpg')).astype('int64')

bf_add = base_face + ci 
bf_sub = base_face - ci 

bf_add[bf_add>255] = 255
bf_sub[bf_sub<0] = 0

# nor = lambda x:(x-x.min())/(x.max() - x.min()) 
# bf_add_x = nor(bf_add) *255
# bf_sub_x = nor(bf_sub) *255
(Image.fromarray(bf_add.astype('uint8'))).save('D:\cnnface\Emotion_analysis\CI_analysis\ci/bf_add.jpg',quality=95)
(Image.fromarray(bf_sub.astype('uint8'))).save('D:\cnnface\Emotion_analysis\CI_analysis\ci/bf_sub.jpg',quality=95)



In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as pg

CI_sinusoid_nor = 'D:/cnnface/female_male_test_51_addnoise\Face_template/classification_noise/CI_sinusoid_nor.jpg'
CI = Image.open(CI_sinusoid_nor)
CI_arr = np.array(CI)

ci_map = plt.imshow(CI_arr,cmap = 'jet',interpolation='nearest')
save_path  ='D:/cnnface/female_male_test_51_addnoise/Face_template/classification_noise/CI_jet.jpg'
pg.imsave(save_path,CI_arr,cmap = 'jet',format='jpg',dpi=1000)




In [ ]:
#load RData file to python
import rpy2.robjects as robjects
import numpy as np
robjects.r['load']("D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/pythonRead/sinusoid1000_patches.RData")

x = robjects.r['patches']
print(np.array(x))

In [10]:
#convert .RData to npy file
import rpy2.robjects as robjects
import numpy as np

robjects.r['load'](r"D:\cnnface\Emotion_analysis\noise_metadata/neu_run1.Rdata")
params = robjects.r['stimuli_params'][0]

# robjects.r['load'](r"D:\cnnface\Identity_analysis\meat_data/run2.Rdata")
# params = robjects.r['stimuli_params'][0]

params_5000_ori = np.array(params)
params_5000_inv = -np.array(params)
params_5000 = np.zeros((5000,4092))
for i in range(2500):
    params_5000[i*2+1,:] = params_5000_ori[i,:]
    params_5000[i*2,:] = params_5000_inv[i,:]
    
#p = np.concatenate((params_1000,params_5000),axis=0)

np.save(r'D:\cnnface\Emotion_analysis\noise_metadata/neu_params_5000',params_5000)

In [ ]:
#convert .RData to npy file
import rpy2.robjects as robjects
import numpy as np

robjects.r['load']("D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/rcic_seed_1_time_9月_18_2019_10_07.Rdata")
p = robjects.r['p']
print(p.names)
patches = p[0]
patchidx = p[1]

patches = np.array(patches)
patchidx = np.array(patchidx)

np.save('D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/patches.npy',patches)
np.save('D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/patchidx.npy',patchidx)